<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
# ! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 
! pip install -q deepmatch

--2022-09-15 20:10:31--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  26.0MB/s    in 0.2s    

2022-09-15 20:10:32 (26.0 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-15 20:10:32--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [4]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

# Data 

In [5]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [6]:
data.user_id.nunique(), data.movie_id.nunique(), data.head(2)

(46, 2175,    user_id  movie_id  rating  timestamp  \
 0        1      1193       5  978300760   
 1        1       661       3  978302109   
 
                                     title     genres gender  age  occupation  \
 0  One Flew Over the Cuckoo's Nest (1975)      Drama      F    1          10   
 1        James and the Giant Peach (1996)  Animation      F    1          10   
 
      zip  
 0  48067  
 1  48067  )

# Feature Engineering

In [7]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
negsample = 0


In [8]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1

In [9]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18
2,1,483,3,978301968,My Fair Lady (1964),12,1,1,9,18
3,1,1881,4,978300275,Erin Brockovich (2000),8,1,1,9,18
4,1,1306,5,978824291,"Bug's Life, A (1998)",3,1,1,9,18


In [10]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')

In [11]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [12]:
train_set, test_set = gen_data_set(data, SEQ_LEN, negsample)

100%|██████████| 46/46 [00:00<00:00, 296.99it/s]

8 8


In [13]:
data.loc[2]

user_id                         1
movie_id                      483
rating                          3
timestamp               978301968
title         My Fair Lady (1964)
genres                         12
gender                          1
age                             1
occupation                      9
zip                            18
Name: 2, dtype: object

In [14]:
print(train_set[0])

# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - constant
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len,  - 50
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating

(20, 1242, 1, [42, 582, 306, 2039, 2021, 698, 1797, 650, 355, 1075, 1293, 1304, 1921, 679, 711, 2154, 1791, 870, 916, 1479, 1438, 648, 27, 604, 416, 629, 329, 790, 595, 470, 1066, 1705, 669, 589, 564, 732, 119, 1434, 504, 726, 1271, 558, 1967, 1970, 468, 773, 1139, 177, 910, 230], 50, [8, 5, 8, 8, 8, 10, 8, 1, 2, 8, 8, 5, 8, 8, 18, 5, 8, 1, 8, 5, 1, 15, 8, 4, 15, 5, 10, 5, 12, 5, 12, 5, 1, 5, 12, 5, 8, 12, 12, 3, 5, 4, 12, 12, 12, 8, 5, 8, 8, 1], 5, 3)


In [15]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((9908,), (9908,), (46,), (46,))

In [16]:
train_model_input

{'user_id': array([20, 13, 29, ..., 21, 35, 43]),
 'movie_id': array([1242,  560, 1234, ..., 1889,  649,  735]),
 'hist_movie_id': array([[  42,  582,  306, ...,  177,  910,  230],
        [1393,    0,    0, ...,    0,    0,    0],
        [ 714,  497,  507, ..., 1546, 1506, 1586],
        ...,
        [ 586, 1796, 1713, ..., 1071, 1188,  740],
        [1285,  774,  601, ..., 1075, 1683,  688],
        [ 257,  908,   44, ...,  303,  711,  517]], dtype=int32),
 'hist_genres': array([[ 8,  5,  8, ...,  8,  8,  1],
        [ 2,  0,  0, ...,  0,  0,  0],
        [ 5, 10,  8, ...,  1,  3, 16],
        ...,
        [ 5,  5, 18, ...,  8,  6,  8],
        [ 5,  8,  8, ...,  8,  6,  1],
        [ 6,  8,  8, ...,  8, 18,  5]], dtype=int32),
 'hist_len': array([50,  1, 50, ..., 50, 50, 50]),
 'genres': array([ 5,  2, 13, ...,  8,  1,  1]),
 'gender': array([1, 2, 2, ..., 1, 2, 2]),
 'age': array([6, 5, 4, ..., 4, 2, 3]),
 'occupation': array([ 2, 14, 16, ...,  4,  5,  6]),
 'zip': array([22, 43, 

# Aggregate Feature and create Embeddings

In [17]:
embedding_dim = 32

In [18]:
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

In [19]:
item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]


# Negative Sampling

In [20]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)

In [22]:
len(sampler_config.item_count)

2176

# Model Training

In [40]:
import deepctr
print(deepctr.__file__)

/usr/local/lib/python3.7/dist-packages/deepctr/__init__.py


In [23]:
import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

In [24]:
model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)


In [25]:

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )


Train on 9908 samples
Epoch 1/20
9908/9908 [==============================] - 3s 351us/sample - loss: 11.4910
Epoch 2/20
9908/9908 [==============================] - 2s 166us/sample - loss: 9.0874
Epoch 3/20
9908/9908 [==============================] - 2s 175us/sample - loss: 8.4495
Epoch 4/20
9908/9908 [==============================] - 1s 135us/sample - loss: 8.2581
Epoch 5/20
9908/9908 [==============================] - 1s 142us/sample - loss: 7.6851
Epoch 6/20
9908/9908 [==============================] - 1s 97us/sample - loss: 7.3220
Epoch 7/20
9908/9908 [==============================] - 1s 68us/sample - loss: 6.7727
Epoch 8/20
9908/9908 [==============================] - 1s 68us/sample - loss: 6.5852
Epoch 9/20
9908/9908 [==============================] - 1s 67us/sample - loss: 6.2486
Epoch 10/20
9908/9908 [==============================] - 1s 67us/sample - loss: 5.9885
Epoch 11/20
9908/9908 [==============================] - 1s 67us/sample - loss: 5.7084
Epoch 12/20
9908/9908 [=

In [26]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(46, 32)
(2175, 32)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [27]:
user_embs

array([[-0.264819  ,  0.05832044, -0.09766247, ...,  0.23743995,
         0.21794611,  0.21923687],
       [-0.3009668 , -0.06624633, -0.08452063, ...,  0.37955567,
         0.06313822,  0.01527649],
       [ 0.17675433,  0.13787495,  0.17622267, ...,  0.24459417,
         0.05199043,  0.05677998],
       ...,
       [-0.12916228,  0.21970025,  0.07554316, ...,  0.3182818 ,
         0.19474615,  0.04992663],
       [-0.03612703, -0.13951112, -0.07517821, ...,  0.5495877 ,
        -0.07213011,  0.0851706 ],
       [-0.05135955, -0.01556315, -0.16766955, ...,  0.2747962 ,
         0.10692497,  0.01283878]], dtype=float32)